In [1]:
#System Imports
import os
import glob


#Math Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import sklearn as sk

#Data Analysis Imports
import scipy.io
import librosa
import librosa.display
import pandas as pd
from pandas import DataFrame

#NN Import
import tensorflow as tf

In [2]:
def get_directories(ROOT_PATH, directory):

    directories = [d for d in os.listdir(directory)
                if os.path.isdir(os.path.join(directory, d))]

    return directories

In [3]:
def load_sound_wave(parent_dir,sub_dirs,file_ext="*.wav"):
    """
    load_sound_wave extracts from the list the amplitude of the audio signal(x)
    and the sampling rate
    :param parent_dir: Location of Whale data for import
    sub_dirs
    file_ext="*.wav"
    :return: x: Audio Signal
             sr: Sampling Rate
    """
    df = DataFrame()
    for l, sub_dir in enumerate(sub_dirs):

        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            sound_clip, sr = librosa.load(fn)
            label = l
            df['t'] = [sound_clip for sound_clip in range(sound_clip)]
            df['label'] = label


    return df, sr, label


print(df)


In [ ]:
def load_data(fn):
    with open(fn, "r") as dill_file:
    files = dill.load( dill_file)
    return files

In [3]:
def one_hot_encode(labels):
    n_labels = labels
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.eye(n_unique_labels)


    return one_hot_encode


In [6]:

def lstm_cell(n_hidden,state_is_tuple = True):
  return tf.contrib.rnn.BasicLSTMCell(n_hidden)

In [7]:

def RNN(x, weight, bias, number_of_layers, n_hidden):
    cell = tf.contrib.rnn.MultiRNNCell(
        [lstm_cell(n_hidden, state_is_tuple=True) for _ in range(number_of_layers)])
    output, state = tf.nn.static_rnn(cell, x, dtype=tf.float32)
    output = tf.transpose(output, [1, 0, 2])
    output = tf.transpose(output, [1, 0, 2])
    last = tf.gather(output, int(output.get_shape()[0]) - 1)
    return tf.nn.softmax(tf.matmul(last, weight) + bias)

In [13]:
def get_data():
    ROOT_PATH = "/home/tim/Documents/Masters/Data"

    train_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Training")
    test_data_directory = os.path.join(ROOT_PATH, "Autoencoder test/Testing")

    train_directories = get_directories(ROOT_PATH, train_data_directory)
    test_directories = get_directories(ROOT_PATH, test_data_directory)


    raw_sounds_tr, sr_tr, tr_labels = load_sound_wave(train_data_directory, train_directories, file_ext="*.wav")
    raw_sounds_ts, sr_ts, ts_labels = load_sound_wave(test_data_directory, test_directories, file_ext="*.wav")
    return raw_sounds_tr, raw_sounds_ts, sr_tr, sr_ts, tr_labels, ts_labels

In [14]:
#def main():
"""

:return:
"""

raw_sounds_tr, raw_sounds_ts, sr_tr, sr_ts, tr_labels, ts_labels = get_data()

######################################
tr_labels = one_hot_encode(tr_labels)
ts_labels = one_hot_encode(ts_labels)

tf.reset_default_graph()

learning_rate = 0.01
training_iters = 1000
batch_size = 50
display_step = 200

# Network Parameters

number_of_layers = 2

n_hidden = 300
n_classes = 2

x = tf.placeholder("float", [None], name='x')
y = tf.placeholder("float", [None, n_classes], name='y')

weight = tf.Variable(tf.random_normal([n_hidden, n_classes]))
bias = tf.Variable(tf.random_normal([n_classes]))
raw_sounds_tr = tf.unstack(raw_sounds_tr)
raw_sounds_ts = tf.unstack(raw_sounds_ts)

prediction = RNN(x, weight, bias, number_of_layers, n_hidden)

# Define loss and optimizer
loss_f = -tf.reduce_sum(y * tf.log(prediction))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss_f)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)

    for itr in range(training_iters):
        offset = (itr * batch_size) % (tr_labels.shape[0] - batch_size)
        batch_x = raw_sounds_tr[offset:(offset + batch_size)]
        batch_x = np.squeeze(batch_x)
        batch_y = tr_labels[offset:(offset + batch_size)]
        _, c = session.run([optimizer, loss_f], feed_dict={x: batch_x, y: batch_y})

        if itr % display_step == 0:
            # Calculate batch accuracy
            acc = session.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            # Calculate batch loss
            loss = session.run(loss_f, feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(epoch) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))

    print('Test accuracy: ', round(session.run(accuracy, feed_dict={x: raw_sounds_ts, y: ts_labels}), 3))




ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/tim/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-a8213998818b>", line 33, in <module>
    raw_sounds_ts = tf.unstack(raw_sounds_ts)
  File "/home/tim/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/array_ops.py", line 975, in unstack
    return gen_array_ops._unpack(value, num=num, axis=axis, name=name)
  File "/home/tim/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gen_array_ops.py", line 3777, in _unpack
    name=name)
  File "/home/tim/anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/op_def_library.py", line 767, in apply_op
    op_def=op_def)
  File "/home/tim/anaconda3/lib/python3.5/site-packages/tensorflow/python/framework/ops.py", line 2508, in create_op
    set_shapes_for_outputs(ret)
KeyboardInterrupt

During handling of the above exception, another exceptio

KeyboardInterrupt: 